In [8]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time

# Get the script

In [2]:
r = requests.get('https://subslikescript.com/series/The_Wire-306414')
r.status_code

200

In [3]:
soup = BeautifulSoup(r.text)

In [28]:
data = {
    'episode': [],
    'title': [],
    'script': []
}

In [29]:
data

{'episode': [], 'title': [], 'script': []}

In [33]:
from tqdm import tqdm

In [34]:
for season in tqdm(soup.find_all('div', {'class':'season'})):
    # print(season.prettify())
    for episode in season.find_all('li'):        
        data['episode'].append(int(episode.text.split('.')[0]))
        data['title'].append(episode.text.split('.')[1])
        
        episode_page = 'https://subslikescript.com' + episode.find('a').get('href')
        request = requests.get(episode_page)
        script_soup = BeautifulSoup(request.text)
        script = script_soup.find('div', {'class':'full-script'}).get_text().replace('-', ' ').replace('?', '.').replace('.', ' ').strip()
        data['script'].append(script)
        
        time.sleep(2)
            
    time.sleep(5)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:09<00:00, 37.81s/it]


In [55]:
data['episode'] = data['episode'][1:]

In [110]:
df_scripts = pd.DataFrame.from_dict(data)
df_scripts.drop(index=[0], inplace=True)
df_scripts.rename(columns=str.capitalize, inplace=True)
df_scripts

,Episode,Title,Script
1,1,The Target,"So, your boy's name is what Snot You called ..."
2,2,The Detail,"(McNulty) That's the witness,the one from the ..."
3,3,The Buys,"Yo, shit is late Yeah, they stillvialin' up Us..."
4,4,Old Cases,(Man straining)Fuck me I'm dying here Let me g...
5,5,The Pager,"'We gonna play the brand newbig, bad game Cal..."
6,6,The Wire,"(Dog barking)(People arguing, distant)(Radio p..."
7,7,One Arrest,"'Low man scrapped, yo He all the way down But ..."
8,8,Lessons,"Number 18 Conine I knew that one All right, th..."
9,9,Game Day,"Him on the ball, from Dunbar He junior college..."
10,10,The Cost,This is the second time this week I had to pay...


# Get the Metadata

In [67]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_The_Wire_episodes")
r.status_code

200

In [68]:
soup = BeautifulSoup(r.text)
# soup

In [69]:
table_html = soup.find('table', {'class':'wikitable plainrowheaders wikiepisodetable'})
table = pd.read_html(str(table_html))

In [71]:
tables = []
for table_html in soup.find_all('table', {'class':'wikitable plainrowheaders wikiepisodetable'}):
#     print(len(tables))
    tables.append(pd.read_html(str(table_html)))

In [81]:
df_meta = pd.DataFrame()

for t in range(len(tables)):
    df_meta = pd.concat([df_meta, tables[t][0]])

In [87]:
df_meta.Title = df_meta.Title.str.replace('"', '')

In [88]:
df_meta

,No. overall,No. in season,Title,Directed by,Written by,Original air date,U.S. viewers (millions),Viewers (millions)
0,1,1,The Target,Clark Johnson,Story by : David Simon & Ed Burns Teleplay by ...,"June 2, 2002",3.70[1],NaN
1,2,2,The Detail,Clark Johnson,Story by : David Simon & Ed Burns Teleplay by ...,"June 9, 2002",2.80[2],NaN
2,3,3,The Buys,Peter Medak,Story by : David Simon & Ed Burns Teleplay by ...,"June 16, 2002",NaN,NaN
3,4,4,Old Cases,Clement Virgo,Story by : David Simon & Ed Burns Teleplay by ...,"June 23, 2002",2.91[3],NaN
4,5,5,The Pager,Clark Johnson,Story by : David Simon & Ed Burns Teleplay by ...,"June 30, 2002",2.97[4],NaN
5,6,6,The Wire,Ed Bianchi,Story by : David Simon & Ed Burns Teleplay by ...,"July 7, 2002",2.98[5],NaN
6,7,7,One Arrest,Joe Chappelle,Story by : David Simon & Ed Burns Teleplay by ...,"July 21, 2002",4.12[6],NaN
7,8,8,Lessons,Gloria Muzio,Story by : David Simon & Ed Burns Teleplay by ...,"July 28, 2002",3.31[7],NaN
8,9,9,Game Day,Milčo Mančevski,Story by : David Simon & Ed Burns Teleplay by ...,"August 4, 2002",3.42[8],NaN
9,10,10,The Cost,Brad Anderson,Story by : David Simon & Ed Burns Teleplay by ...,"August 11, 2002",4.15[9],NaN


# Merge the 2 dfs

In [119]:
df = df_meta.copy()
df.set_index('No. overall', inplace=True)

In [120]:
df

,No. in season,Title,Directed by,Written by,Original air date,U.S. viewers (millions),Viewers (millions)
No. overall,,,,,,,
1,1,The Target,Clark Johnson,Story by : David Simon & Ed Burns Teleplay by ...,"June 2, 2002",3.70[1],NaN
2,2,The Detail,Clark Johnson,Story by : David Simon & Ed Burns Teleplay by ...,"June 9, 2002",2.80[2],NaN
3,3,The Buys,Peter Medak,Story by : David Simon & Ed Burns Teleplay by ...,"June 16, 2002",NaN,NaN
4,4,Old Cases,Clement Virgo,Story by : David Simon & Ed Burns Teleplay by ...,"June 23, 2002",2.91[3],NaN
5,5,The Pager,Clark Johnson,Story by : David Simon & Ed Burns Teleplay by ...,"June 30, 2002",2.97[4],NaN
6,6,The Wire,Ed Bianchi,Story by : David Simon & Ed Burns Teleplay by ...,"July 7, 2002",2.98[5],NaN
7,7,One Arrest,Joe Chappelle,Story by : David Simon & Ed Burns Teleplay by ...,"July 21, 2002",4.12[6],NaN
8,8,Lessons,Gloria Muzio,Story by : David Simon & Ed Burns Teleplay by ...,"July 28, 2002",3.31[7],NaN
9,9,Game Day,Milčo Mančevski,Story by : David Simon & Ed Burns Teleplay by ...,"August 4, 2002",3.42[8],NaN


In [121]:
df['Script'] = df_scripts.Script.to_list()
df['Episode'] = df_scripts.Episode.to_list()

In [127]:
# create season column
df['Season'] = [1]*13 + [2]*12 + [3]*12 + [4]*13 + [5]*10

In [147]:
df[df['Season'] == 5]

,No. in season,Title,Directed by,Original air date,U.S. viewers (millions),Viewers (millions),Script,Episode,Season,Story Writer,Teleplay Writer
No. overall,,,,,,,,,,,
51,1,More with Less,Joe Chappelle,"January 6, 2008",1.23[33],NaN,"You see now I'm here to tell you,this remainin...",1,5,David Simon & Ed Burns,David Simon
52,2,Unconfirmed Reports,Ernest Dickerson,"January 13, 2008",1.19[34],NaN,"You know how that go, right Gave myself all th...",2,5,David Simon & William F. Zorzi,William F. Zorzi
53,3,Not for Attribution,Scott Kecken & Joy Kecken,"January 20, 2008",0.85[35],NaN,"Type quieter, asshole Early relief So get the ...",3,5,David Simon & Chris Collins,Chris Collins
54,4,Transitions,Dan Attias,"January 27, 2008",1.28[36],NaN,Fuck is he doing Cops still parked right up th...,4,5,David Simon & Ed Burns,Ed Burns
55,5,React Quotes,Agnieszka Holland,"February 3, 2008",0.53[37],NaN,Joe will be missed Very much missed We liked ...,5,5,David Simon & David Mills,David Mills
56,6,The Dickensian Aspect,Seith Mann,"February 10, 2008",0.74[38],NaN,Gotta be somewhere You wanna keep goin' Office...,6,5,David Simon & Ed Burns,Ed Burns
57,7,Took,Dominic West,"February 17, 2008",0.57[39],NaN,Now they're online This is maskedas Marlo's ce...,7,5,David Simon & Richard Price,Richard Price
58,8,Clarifications,Anthony Hemingway,"February 24, 2008",NaN,NaN,"And as you can seefrom the locations,the kille...",8,5,David Simon & Dennis Lehane,Dennis Lehane
59,9,Late Editions,Joe Chappelle,"March 2, 2008",0.71[40],NaN,"23 16 Yeah, Lester Where are you at On M...",9,5,David Simon & George Pelecanos,George Pelecanos


In [142]:
df['Teleplay Writer'] = df['Written by'].str.split(' Teleplay by : ', expand=True)[1]
df['Story Writer'] = df['Written by'].str.split(' Teleplay by : ', expand=True)[0].str.replace('Story by : ', '')
df.drop(columns=['Written by'], inplace=True)

In [143]:
df

,No. in season,Title,Directed by,Original air date,U.S. viewers (millions),Viewers (millions),Script,Episode,Season,Story Writer,Teleplay Writer
No. overall,,,,,,,,,,,
1,1,The Target,Clark Johnson,"June 2, 2002",3.70[1],NaN,"So, your boy's name is what Snot You called ...",1,1,David Simon & Ed Burns,David Simon
2,2,The Detail,Clark Johnson,"June 9, 2002",2.80[2],NaN,"(McNulty) That's the witness,the one from the ...",2,1,David Simon & Ed Burns,David Simon
3,3,The Buys,Peter Medak,"June 16, 2002",NaN,NaN,"Yo, shit is late Yeah, they stillvialin' up Us...",3,1,David Simon & Ed Burns,David Simon
4,4,Old Cases,Clement Virgo,"June 23, 2002",2.91[3],NaN,(Man straining)Fuck me I'm dying here Let me g...,4,1,David Simon & Ed Burns,David Simon
5,5,The Pager,Clark Johnson,"June 30, 2002",2.97[4],NaN,"'We gonna play the brand newbig, bad game Cal...",5,1,David Simon & Ed Burns,Ed Burns
6,6,The Wire,Ed Bianchi,"July 7, 2002",2.98[5],NaN,"(Dog barking)(People arguing, distant)(Radio p...",6,1,David Simon & Ed Burns,David Simon
7,7,One Arrest,Joe Chappelle,"July 21, 2002",4.12[6],NaN,"'Low man scrapped, yo He all the way down But ...",7,1,David Simon & Ed Burns,Rafael Alvarez
8,8,Lessons,Gloria Muzio,"July 28, 2002",3.31[7],NaN,"Number 18 Conine I knew that one All right, th...",8,1,David Simon & Ed Burns,David Simon
9,9,Game Day,Milčo Mančevski,"August 4, 2002",3.42[8],NaN,"Him on the ball, from Dunbar He junior college...",9,1,David Simon & Ed Burns,David H. Melnick & Shamit Choksey


In [149]:
# save to file
df.to_csv('the wire script.csv', index=False)